In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Subset, ChainDataset, DataLoader, random_split
import torch.nn.functional as F

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True

import pandas as pd
import pickle
import math
import random

import HSIC
import matplotlib.pyplot as plt
import numpy as np
import func
import torchy

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter


from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
import wandb

%matplotlib widget

In [2]:
data_root = "../../Data/PACS/"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# print(dict(Counter(dataset.targets)))
num_classes = 7      # 7 classes for each domain: 'dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person'
all_domain_names = ['photo', 'art_painting', 'cartoon', 'sketch']
domain_names = ['photo', 'art_painting', 'cartoon']
classes_names = ['Dog', 'Elephant', 'Giraffe', 'Guitar', 'Horse', 'House', 'Person']
domain_mapping = {'photo':0, 'art_painting':1, 'cartoon':2, 'sketch':3}
model_path = 'saved_model.pth'


train_names = ['photo', 'art_painting']
valid_name = ['cartoon']
test_name = 'split'

# means and standard deviations ImageNet because the network is pretrained
means, stds = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

# Define transforms to apply to each image
transf = transforms.Compose([ #transforms.Resize(227),      # Resizes short size of the PIL image to 256
                            transforms.CenterCrop(224),  # Crops a central square patch of the image 224 because torchvision's AlexNet needs a 224x224 input!
                            transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                            transforms.Normalize(means,stds) # Normalizes tensor with mean and standard deviation
])

datasets = {}

for name in os.listdir(data_root):

    if not name[0] == '.':
        dataset = torchvision.datasets.ImageFolder(data_root+name, transform=transf)

        datasets[name] = dataset
        print(f"Added :{name}, length: {len(datasets[name])}")


Added :cartoon, length: 2344
Added :art_painting, length: 2048
Added :photo, length: 1670
Added :sketch, length: 3929


In [3]:
# gc.collect() # Python thing
# torch.cuda.empty_cache() # PyTorch thing


criterion = nn.CrossEntropyLoss()

train_names = ['art_painting', 'cartoon']
valid_name = 'split'
test_name = ['photo']

lr=0.0001
use_hsic = True
batch_size = 64
epochs = 5
verbose = True
now = datetime.now()
current_time = now.strftime("%H_%M_%S")


for gamma in [0.2]:
    print('gamma: ', gamma)
    for i in range(2):
        print(i)
        
        current_time = now.strftime("%H_%M_%S")
#         wandb.init(project=f"colab_runs_embeddings",
#                     entity="skohnie",
#                     name=f'{current_time}/{gamma}/{i}',
#                     config = {"learning_rate": lr,
#                                 "epochs": epochs,
#                                 "batch_size": batch_size,
#                                 "gamma": gamma,
#                                 "Train names": train_names,
#                                 "Valid name": valid_name,
#                                 "Test name": test_name
#                                 }
#                 )



        train_loader, valid_loader, test_loader  = torchy.get_image_loaders(datasets, 
                                                                            batch_size,
                                                                            train_names,
                                                                            valid_name,
                                                                            test_name,
                                                                            verbose=verbose)


        resnet18 = models.resnet18(pretrained=True)
        net = torchy.CustomNet(resnet18)


        min_valid_loss = 1000



        optimizer = optim.Adam(net.parameters(), lr=lr) 
        min_valid_loss = func.train(net, criterion, optimizer, 
                                    train_loader,
                                    valid_loader=valid_loader,
                                    epochs=epochs,
                                    use_hsic=use_hsic,
                                    gamma=gamma,
                                    device=device,
                                    writer=None,
                                    min_valid_loss = min_valid_loss,
                                    wb=False,
                                    verbose=verbose)



        _, _, acc = func.test_model(net, test_loader, 'saved_model.pth', verbose=True)
        print(acc)
#         wandb.log({'Testing accuracy': acc})
#         wandb.summary['Test Accuracy'] = acc
#         wandb.finish()

gamma:  0.2
0
Training on ['art_painting', 'cartoon']
Validation on trainingset split
Testing on ['photo']
Using HSIC:True
Cuda:cpu
Epoch 0
HSIC: tensor(0.6603, grad_fn=<DivBackward0>)
Training loss: tensor(4.8645, grad_fn=<AddBackward0>)
Validation Loss Decreased(1000.000000--->163.408525) 	 Saving The Model


KeyboardInterrupt: 

In [ ]:
NUM_CLASSES = 7      # 7 classes for each domain: 'dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person'
DOMAIN_NAMES = ['photo', 'art_painting', 'cartoon', 'sketch']
CLASSES_NAMES = ['Dog', 'Elephant', 'Giraffe', 'Guitar', 'Horse', 'House', 'Person']
domain_mapping = {'photo':0, 'art_painting':1, 'cartoon':2, 'sketch':3}
batch_size = 64
PATH = 'saved_model.pth'

criterion = nn.CrossEntropyLoss()


lambd = 1
i = 0

# train_names = [DOMAIN_NAMES[1]]
# valid_name = DOMAIN_NAMES[2]
# test_name = DOMAIN_NAMES[0]
# hsic_name =  DOMAIN_NAMES[3]

train_names = DOMAIN_NAMES[1:]
valid_name = 'split'
test_name = DOMAIN_NAMES[0]
hsic_name =  None

train_loader, valid_loader, test_loader, HSIC_loader  = torchy.get_image_loaders(datasets,
                                                                           batch_size,
                                                                           train_names,
                                                                           hsic_name,
                                                                           valid_name,
                                                                           test_name)


# Display image and label.
train_features, train_labels, _ = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
images = train_features
plt.imshow(torchvision.utils.make_grid(images, nrow=5).permute(1, 2, 0))
plt.show()
print(f"Label: {train_labels}")

In [ ]:
    
acc_list = []                          
min_valid_loss = 1000
for lr in [0.02, 0.001]:
    
    resnet18 = models.resnet18(pretrained=True)
    resnet18.fc1 = nn.Linear(512, 7)
    
    writer= SummaryWriter(f'runs/test_vis')
    optimizer = optim.Adam(resnet18.parameters(), lr=lr)
    min_valid_loss = func.train(resnet18, criterion, optimizer, train_loader,
                                    valid_loader=valid_loader,
                                    epochs=20,
                                    HSIC_loader=HSIC_loader,
                                    lambd=lambd,
                                    writer=writer,
                                    min_valid_loss = min_valid_loss)
    writer.close()
    